In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

sparkSession = SparkSession.builder.master("spark://winemap:7077").getOrCreate()

In [ ]:
import psycopg2
conn = psycopg2.connect("host='172.17.0.4' port='5432' dbname='wineDb' user='username' password='password'")
cur = conn.cursor()
#make table
f = open(r'wineData.csv', 'r')
cur.copy_from(f, "wine_reviews", sep=',')
conn.commit()
f.close()

In [ ]:
url = "jdbc:postgresql://172.17.0.4/wineDb"

(sparkSession.read.format("jdbc")
    .options(url=url, dbtable="wine_review")
    .load())
